# Financial Prediction

## Do The imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import yfinance as yf
import riskfolio as rpf
from prophet import Prophet
import gradio

## Gradio Input

In [ ]:
start = "2012-01-01" #input gradio
end = "2019-12-31"  #input gradio
tickers_list = ["TSLA","GOOGL","AAPL"]  #input gradio
analyze_start_date = "2018-01-01" #input gradio
dict_resample = {"yearly" : "Y", "monthly" : "M", "weekly" : "W","daily" : "D"}
resample_period = "monthly" #input gradio
periods = 24    #input gradio
ticker_market_price_chart = "AAPL"  #input gradio

## Download Data

In [ ]:
data = yf.download(tickers_list, start = start, end = end)
data = data.loc[:,("Adj Close")]
data = data.reset_index()

## Prepare the Data

In [ ]:
analyze_start_data = data[data.index == data[data["Date"] <= analyze_start_date].index[-1]]
data_resample = data.resample(dict_resample[resample_period], on = "Date").last()
data_resample = data_resample.dropna()
data_resample = data_resample.reset_index()

data_tickers_optimisation_past_save = data_resample[data_resample["Date"]<analyze_start_date]
data_tickers_optimisation_overall_save = data_resample.copy()

data_tickers_prediction_optimisation_overall_save = pd.DataFrame()

## Make Predictions

In [ ]:
for ticker in tickers_list : 

    data_ticker_model = data_resample[data_resample["Date"] < analyze_start_date][["Date",ticker]].rename(columns = {"Date" : "ds", ticker : "y"})

    model = Prophet()
    model.fit(data_ticker_model)
    futur = model.make_future_dataframe(periods = periods , freq = dict_resample[resample_period])
    data_ticker_prediction = model.predict(futur)
    
    #model.plot(data_ticker_prediction,uncertainty=True)
    #model.plot_components(data_ticker_prediction)

    data_ticker_prediction = data_ticker_prediction[data_ticker_prediction["ds"] >= analyze_start_date][["ds","yhat"]].rename(columns = {"ds" : "Date", "yhat": f"{ticker} prediction"})

    data_ticker_prediction = pd.concat([data_tickers_optimisation_past_save.rename(columns = { ticker : f"{ticker} prediction"})[["Date",f"{ticker} prediction"]],
                                        data_ticker_prediction], ignore_index = True)

    if "Date" in data_tickers_prediction_optimisation_overall_save.columns : 
        data_tickers_prediction_optimisation_overall_save[f"{ticker} prediction"] = data_ticker_prediction[f"{ticker} prediction"]
    else :
        data_tickers_prediction_optimisation_overall_save = data_ticker_prediction

## Draw Ticker Market Price Chart

In [ ]:
plt.figure(figsize = (10,6))
plt.title(f"Market Price : {ticker_market_price_chart} and {ticker_market_price_chart} prediction evolution")
plt.axvline(x = datetime.datetime.strptime(analyze_start_date,"%Y-%M-%d"), color = "black", linestyle = "dashed")
plt.plot(data_tickers_prediction_optimisation_overall_save["Date"],data_tickers_prediction_optimisation_overall_save[f"{ticker_market_price_chart} prediction"],
        color="red",label=f"{ticker_market_price_chart} prediction")
plt.plot(data_resample["Date"],data_resample[ticker_market_price_chart],color = "blue", label = ticker_market_price_chart)
plt.yscale(value = "log")
plt.xlabel("Time")
plt.ylabel("Price (log[10] scale)")
plt.legend(loc = "upper left")
plt.show()

## Portfolio Tickers Past Optimisation 

In [ ]:
data_tickers_optimisation_past = data_tickers_optimisation_past_save.copy()
data_tickers_optimisation_past[data_tickers_optimisation_past.columns.drop("Date")] = data_tickers_optimisation_past[data_tickers_optimisation_past.columns.drop("Date")].pct_change()
data_tickers_optimisation_past = data_tickers_optimisation_past.dropna()
data_tickers_optimisation_past = data_tickers_optimisation_past.set_index("Date")

portfolio = rpf.Portfolio(returns = data_tickers_optimisation_past)
method_mu="hist" 
method_cov="hist"
portfolio.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.95)
model="Classic" # + "BL" --> list and loop over
rm = "MV" # + "SLPM" --> list and loop over
obj = "Sharpe"
hist = True
rf = 0
l = 0
tickers_weight = portfolio.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
ax1 = rpf.plot_pie(w=tickers_weight, title="Sharpe Mean Variance", others=0.01, nrow=20, cmap = "tab20", height=6, width=10, ax=None)

In [ ]:
points = 50
frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)
label = 'Max Risk Adjusted Return Portfolio'
mu = portfolio.mu
cov = portfolio.cov
returns = portfolio.returns
ax2 = rpf.plot_frontier(w_frontier=frontier, mu=mu, cov=cov, returns=returns, rm=rm, rf=rf, alpha=0.05, cmap='viridis', w=tickers_weight, 
                        label=label, marker='*', s=16, c='r', height=6, width=10, ax=None)

In [ ]:
ax3 = rpf.plot_frontier_area(w_frontier=frontier, cmap="tab20", height=6, width=10, ax=None)

## Portfolio Tickers Prediction Optimisation

In [ ]:
data_tickers_prediction_optimisation_overall = data_tickers_prediction_optimisation_overall_save.copy()
data_tickers_prediction_optimisation_overall[data_tickers_prediction_optimisation_overall.columns.drop("Date")] = data_tickers_prediction_optimisation_overall[data_tickers_prediction_optimisation_overall.columns.drop("Date")].pct_change()
data_tickers_prediction_optimisation_overall = data_tickers_prediction_optimisation_overall.dropna()
data_tickers_prediction_optimisation_overall = data_tickers_prediction_optimisation_overall.set_index("Date")

portfolio = rpf.Portfolio(returns = data_tickers_prediction_optimisation_overall)
method_mu="hist" 
method_cov="hist"
portfolio.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.95)
model="Classic" # + "BL" --> list and loop over
rm = "MV" # + "SLPM" --> list and loop over
obj = "Sharpe"
hist = True
rf = 0
l = 0
tickers_weight = portfolio.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
ax1 = rpf.plot_pie(w=tickers_weight, title="Sharpe Mean Variance", others=0.01, nrow=20, cmap = "tab20", height=6, width=10, ax=None)

In [ ]:
points = 50
frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)
label = 'Max Risk Adjusted Return Portfolio'
mu = portfolio.mu
cov = portfolio.cov
returns = portfolio.returns
ax2 = rpf.plot_frontier(w_frontier=frontier, mu=mu, cov=cov, returns=returns, rm=rm, rf=rf, alpha=0.05, cmap='viridis', w=tickers_weight, 
                        label=label, marker='*', s=16, c='r', height=6, width=10, ax=None)

In [ ]:
ax3 = rpf.plot_frontier_area(w_frontier=frontier, cmap="tab20", height=6, width=10, ax=None)

## Portfolio Tickers Overall Optimisation 

In [ ]:
data_tickers_optimisation_overall = data_tickers_optimisation_overall_save.copy()
data_tickers_optimisation_overall[data_tickers_optimisation_overall.columns.drop("Date")] = data_tickers_optimisation_overall[data_tickers_optimisation_overall.columns.drop("Date")].pct_change()
data_tickers_optimisation_overall = data_tickers_optimisation_overall.dropna()
data_tickers_optimisation_overall = data_tickers_optimisation_overall.set_index("Date")

portfolio = rpf.Portfolio(returns = data_tickers_optimisation_overall)
method_mu="hist" 
method_cov="hist"
portfolio.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.95)
model="Classic" # + "BL" --> list and loop over
rm = "MV" # + "SLPM" --> list and loop over
obj = "Sharpe"
hist = True
rf = 0
l = 0
tickers_weight = portfolio.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
ax1 = rpf.plot_pie(w=tickers_weight, title="Sharpe Mean Variance", others=0.01, nrow=20, cmap = "tab20", height=6, width=10, ax=None)

In [ ]:
points = 50
frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)
label = 'Max Risk Adjusted Return Portfolio'
mu = portfolio.mu
cov = portfolio.cov
returns = portfolio.returns
ax2 = rpf.plot_frontier(w_frontier=frontier, mu=mu, cov=cov, returns=returns, rm=rm, rf=rf, alpha=0.05, cmap='viridis', w=tickers_weight, 
                        label=label, marker='*', s=16, c='r', height=6, width=10, ax=None)

In [ ]:
ax3 = rpf.plot_frontier_area(w_frontier=frontier, cmap="tab20", height=6, width=10, ax=None)